In [2]:
import pandas as pd

In [3]:
df = pd.read_pickle('../data/Truck_ride.pkl')

In [4]:
df

,vin,triggerType_triggerType,triggerType_context,createdDateTime,receivedDateTime,hrTotalVehicleDistance,totalEngineHours,grossCombinationVehicleWeight,snapshotData_gnssPosition_latitude,snapshotData_gnssPosition_longitude,...,snapshotData_wheelBasedSpeed,snapshotData_fuelType,snapshotData_fuelLevel1,snapshotData_catalystFuelLevel,snapshotData_driver1WorkingState,snapshotData_driver2WorkingState,snapshotData_ambientAirTemperature,uptimeData_engineCoolantTemperature,uptimeData_serviceBrakeAirPressureCircuit1,uptimeData_serviceBrakeAirPressureCircuit2
0,XLRTEF5300G450805,IGNITION_ON,RFMS,2024-03-27T05:30:02.334Z,2024-03-27T05:31:20.694Z,83648285,0,0,51.264967,4.283523,...,0.0,Diesel,0.0,99,REST,REST,20,21,2040,2040
1,XLRTEF5300G450805,DRIVER_1_WORKING_STATE_CHANGED,RFMS,2024-03-27T05:30:19.634Z,2024-03-27T05:31:35.615Z,83648285,0,0,51.264968,4.283523,...,0.0,Diesel,99.2,0,REST,REST,0,0,0,0
2,XLRTEF5300G450805,TIMER,RFMS,2024-03-27T05:31:00.010Z,2024-03-27T05:32:09.086Z,83648285,1703,655350,51.264968,4.283523,...,0.0,Diesel,99.2,99,WORK,REST,7,21,528,528
3,XLRTEF5300G450805,ENGINE_ON,RFMS,2024-03-27T05:31:00.981Z,2024-03-27T05:32:10.642Z,83648285,0,655350,51.264968,4.283523,...,0.0,Diesel,99.2,99,WORK,REST,7,21,528,528
4,XLRTEF5300G450805,TIMER,RFMS,2024-03-27T05:33:44.881Z,2024-03-27T05:34:54.411Z,83648290,1703,655350,51.264961,4.283535,...,4.0,Diesel,99.2,99,WORK,REST,7,29,944,944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,XLRTEF5300G450805,DRIVER_1_WORKING_STATE_CHANGED,RFMS,2024-06-04T07:29:51.710Z,2024-06-04T07:30:26.081Z,112132250,0,0,47.837051,7.395467,...,0.0,Diesel,25.2,0,WORK,DRIVER_AVAILABLE,0,0,0,0
3496,XLRTEF5300G450805,TIMER,RFMS,2024-06-04T07:29:55.060Z,2024-06-04T07:30:30.192Z,112132245,2161,38800,47.837001,7.395462,...,2.0,Diesel,25.2,80,DRIVE,DRIVER_AVAILABLE,13,84,1000,1000
3497,XLRTEF5300G450805,IGNITION_OFF,RFMS,2024-06-04T07:29:55.844Z,2024-06-04T07:30:31.691Z,0,0,0,47.837100,7.395470,...,0.0,Diesel,25.2,102,None,None,13,83,0,0
3498,XLRTEF5300G450805,ENGINE_OFF,RFMS,2024-06-04T07:29:58.481Z,2024-06-04T07:30:32.300Z,0,0,0,47.837050,7.395468,...,0.0,Diesel,25.2,102,None,None,13,83,0,0


In [ ]:
import base64
import requests 
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from sklearn.cluster import KMeans
from sklearn.covariance import EllipticEnvelope
from sklearn.decomposition import PCA
from datetime import datetime, timedelta

warnings.filterwarnings('ignore')


def get_truck_info(vin="XLRTEF5300G450805", start_date="2024-03-25", end_date="2024-06-03"):

    auth_URL = 'https://api.connect.daf.com/rfms/token'
    base_URL = 'https://api.connect.daf.com'

    username= "Hackathon_4@daftrucks.com"
    password = "Team4Hackathon!"
    creds = username +':' + password
    creds_bytes = creds.encode('ascii')
    base64_bytes = base64.b64encode(creds_bytes)
    base64_message = base64_bytes.decode('ascii')
    headers = {
    'Authorization': 'Basic ' + base64_message }

    r = requests.post(auth_URL, headers=headers, verify=False)
    jsondata = r.json()
    token = jsondata.get('access_token')

    datetype ='&datetype=received'
    triggerfilter = '&Triggerfilter=DRIVER_1_WORKING_STATE_CHANGED'

    header = {
    'Authorization': 'Bearer ' + token,
    'Accept': 'application/vnd.fmsstandard.com.vehiclestatuses.v3.0+json; UTF-8'}

    data = []
    current_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    
    while current_date <= end_date:
        starttime = current_date.strftime("%Y-%m-%d") + "T23:59:59.999Z"
        stoptime = (current_date + timedelta(days=1)).strftime("%Y-%m-%d") + "T23:59:59.999Z"

        myurl = base_URL+'/rfms/vehiclestatuses?vin=' + vin +'&starttime=' + starttime

        req = requests.get(myurl, headers=header, verify=False)
        # print(jsondata)
        jsondata=req.json()
        print(jsondata)



        for i in jsondata['vehicleStatusResponse']['vehicleStatuses']:
            dat=[]
            dat.append(i['vin'])
            dat.append(i['triggerType']['triggerType'])
            dat.append(i['triggerType']['context'])
            dat.append(i['createdDateTime'])
            dat.append(i['receivedDateTime'])
            try:
                dat.append(i['hrTotalVehicleDistance'])
            except Exception:
                dat.append(0)

            try:
                dat.append(i['totalEngineHours'])
            except Exception:
                dat.append(0)

            try:
                dat.append(i['grossCombinationVehicleWeight'])
            except Exception:
                dat.append(0)

            try:
                dat.append(i['snapshotData']['gnssPosition']['latitude'])
            except Exception:
                dat.append(-9999)

            try:
                dat.append(i['snapshotData']['gnssPosition']['longitude'])
            except Exception:
                dat.append(-9999)

            try:
                dat.append(i['snapshotData']['gnssPosition']['heading'])
            except Exception:
                dat.append(-9999)

            try:
                dat.append(i['snapshotData']['gnssPosition']['speed'])
            except Exception:
                dat.append(-9999)

            try:
                dat.append(i['snapshotData']['gnssPosition']['positionDateTime'])
            except Exception:
                dat.append(None)

            try:
                dat.append(i['snapshotData']['wheelBasedSpeed'])
            except Exception:
                dat.append(0)

            dat.append(i['snapshotData']['fuelType'])

            try:
                dat.append(i['snapshotData']['fuelLevel1'])
            except Exception:
                dat.append(0)

            try:
                dat.append(i['snapshotData']['catalystFuelLevel'])
            except Exception:
                dat.append(0)

            try:
                dat.append(i['snapshotData']['driver1WorkingState'])
            except Exception:
                dat.append(None)

            try:
                dat.append(i['snapshotData']['driver2WorkingState'])
            except Exception:
                dat.append(None)

            try:
                dat.append(i['snapshotData']['ambientAirTemperature'])
            except Exception:
                dat.append(0)

            try:
                dat.append(i['uptimeData']['engineCoolantTemperature'])
            except Exception:
                dat.append(0)
            try:
                dat.append(i['uptimeData']['serviceBrakeAirPressureCircuit1'])
            except Exception:
                dat.append(0)

            try:
                dat.append(i['uptimeData']['serviceBrakeAirPressureCircuit2'])
            except Exception:
                dat.append(0)

            data.append(dat)

        current_date += timedelta(days=1)

    df = pd.DataFrame(data, columns=['vin', 'triggerType_triggerType',
                                        'triggerType_context','createdDateTime',"receivedDateTime",
                                        "hrTotalVehicleDistance","totalEngineHours","grossCombinationVehicleWeight",
                                        'snapshotData_gnssPosition_latitude','snapshotData_gnssPosition_longitude',
                                        "snapshotData_gnssPosition_heading","snapshotData_gnssPosition_speed",
                                        "snapshotData_gnssPosition_positionDateTime",
                                        'snapshotData_wheelBasedSpeed','snapshotData_fuelType',
                                        'snapshotData_fuelLevel1','snapshotData_catalystFuelLevel',
                                        'snapshotData_driver1WorkingState','snapshotData_driver2WorkingState',
                                        'snapshotData_ambientAirTemperature','uptimeData_engineCoolantTemperature',
                                        'uptimeData_serviceBrakeAirPressureCircuit1','uptimeData_serviceBrakeAirPressureCircuit2'])

    df.drop_duplicates(inplace=True)
    
    return df


def get_dates_online():
    
    # Get today's date
    today = datetime.today().date()

    # Calculate the date 70 days before today
    days_before = today - timedelta(days=70)

    # Format the dates as YYYY-MM-DD
    today_str = today.strftime('%Y-%m-%d')
    days_before_str = days_before.strftime('%Y-%m-%d')

    # Create the tuple
    return today_str, days_before_str

# get the dates
end_date, start_date = get_dates_online()

df_truck = get_truck_info(start_date=start_date, end_date=end_date)

In [5]:
import plotly.graph_objs as go
import pandas as pd
from datetime import datetime
from geopy.distance import geodesic

# Function to join two traces
def join_traces(trace_indices, traces):
    joined_trace = {
        "lon": [],
        "lat": [],
        "marker_color": traces[trace_indices[0]]["marker"]["color"],
        "line_color": traces[trace_indices[0]]["line"]["color"],
    }
    for index in trace_indices:
        trace = traces[index]
        joined_trace["lon"].extend(trace["lon"])
        joined_trace["lat"].extend(trace["lat"])
    return joined_trace

# Load data from CSV
df = pd.read_pickle('../data/trace_demo.pkl')

# Extract latitude and longitude data and filter out invalid coordinates
latitudes = df["snapshotData_gnssPosition_latitude"]
longitudes = df["snapshotData_gnssPosition_longitude"]
valid_coords = [(lat, lon) for lat, lon in zip(latitudes, longitudes) if lat != -9999.0 and lon != -9999.0]

# Calculate distance between consecutive valid points
distances = [0]  # Start with 0 distance for the first point
for i in range(1, len(valid_coords)):
    coord1 = valid_coords[i - 1]
    coord2 = valid_coords[i]
    distances.append(geodesic(coord1, coord2).kilometers)

# Extract time data and filter out invalid timestamps
timestamps = df["snapshotData_gnssPosition_positionDateTime"]
valid_timestamps = []
for ts in timestamps:
    try:
        valid_timestamps.append(datetime.strptime(str(ts), "%Y-%m-%dT%H:%M:%S.%fZ"))
    except ValueError:
        pass  # Skip invalid timestamps

# Define thresholds for distance and time differences to decide whether to start a new trace
distance_threshold = 110  # Adjust this threshold as needed
time_threshold = 60 * 60  # 1 hour (in seconds), adjust as needed

# Create a list to store the traces for different journeys
traces = []

# Initialize color index
color_index = 0

# Iterate through the timestamps and create traces for each journey
journey_trace = []
prev_timestamp = valid_timestamps[0]
for i, (timestamp, coord, distance) in enumerate(zip(valid_timestamps, valid_coords, distances)):
    time_diff = (timestamp - prev_timestamp).total_seconds()
    if time_diff > time_threshold or distance > distance_threshold:
        # Add the current journey trace to the list of traces
        if "REST" in str(df.iloc[i-1]["snapshotData_driver1WorkingState"]):
            point_color = "blue"
        else:
            point_color = f"rgb({color_index * 50}, {255 - color_index * 50}, 0)"
        traces.append(go.Scattermapbox(
            mode="lines+markers",
            lon=[point[1] for point in journey_trace],
            lat=[point[0] for point in journey_trace],
            marker=dict(
                size=10,
                color=point_color,
            ),
            line=dict(
                color=point_color,
                width=2,
            ),
        ))
        # Start a new journey trace
        journey_trace = []
        color_index = (color_index + 1) % 5  # Cycle through 5 colors
    journey_trace.append(coord)
    prev_timestamp = timestamp

# Add the last journey trace to the list of traces
if "REST" in str(df.iloc[-1]["snapshotData_driver1WorkingState"]):
    point_color = "blue"
else:
    point_color = f"rgb({color_index * 50}, {255 - color_index * 50}, 0)"
traces.append(go.Scattermapbox(
    mode="lines+markers",
    lon=[point[1] for point in journey_trace],
    lat=[point[0] for point in journey_trace],
    marker=dict(
        size=10,
        color=point_color,
    ),
    line=dict(
        color=point_color,
        width=2,
    ),
))

# Create layout for the map
layout = go.Layout(
    height=600, width=1000,
    autosize=True,
    hovermode="closest",
    mapbox=dict(
        center=dict(
            lat=52.1326,
            lon=5.2913,
        ),
        style="open-street-map",
        zoom=5,
    ),
)

# Plot the initial map
fig = go.Figure(data=traces, layout=layout)
fig.show()

# Input prompt for joining traces
while True:
    join_indices = input("Enter the indices of the two traces you want to join (comma-separated, e.g., '0,1'), or enter 'q' to quit: ")
    if join_indices.lower() == "q":
        break
    try:
        index1, index2 = map(int, join_indices.split(","))
        if index1 < 0 or index1 >= len(traces) or index2 < 0 or index2 >= len(traces):
            print("Invalid indices. Please enter valid indices.")
        else:
            # Join the selected traces
            joined_trace = join_traces([index1, index2], traces)
            # Remove the original traces
            traces = [trace for i, trace in enumerate(traces) if i not in [index1, index2]]
            # Append the joined trace
            traces.append(go.Scattermapbox(
                mode="lines+markers",
                lon=joined_trace["lon"],
                lat=joined_trace["lat"],
                marker=dict(
                    size=10,
                    color=joined_trace["marker_color"],
                ),
                line=dict(
                    color=joined_trace["line_color"],
                    width=2,
                ),
            ))
            # Update the plot
            fig = go.Figure(data=traces, layout=layout)
            fig.show()
    except ValueError:
        print("Invalid input format. Please enter comma-separated indices (e.g., '0,1').")


In [6]:
df

,vin,triggerType_triggerType,triggerType_context,createdDateTime,receivedDateTime,hrTotalVehicleDistance,totalEngineHours,grossCombinationVehicleWeight,snapshotData_gnssPosition_latitude,snapshotData_gnssPosition_longitude,...,snapshotData_fuelType,snapshotData_fuelLevel1,snapshotData_catalystFuelLevel,snapshotData_driver1WorkingState,snapshotData_driver2WorkingState,snapshotData_ambientAirTemperature,uptimeData_engineCoolantTemperature,uptimeData_serviceBrakeAirPressureCircuit1,uptimeData_serviceBrakeAirPressureCircuit2,vehicle_status
2900,XLRTEF5300G450805,DRIVER_1_WORKING_STATE_CHANGED,RFMS,2024-05-24T03:00:20.743Z,2024-05-24T03:05:24.919Z,106460250,0,0,47.660738,-2.971431,...,Diesel,66.8,0,DRIVER_AVAILABLE,REST,0,0,0,0,At Stop
2901,XLRTEF5300G450805,ENGINE_ON,RFMS,2024-05-24T02:59:51.524Z,2024-05-24T03:05:27.099Z,106460250,0,655350,-9999.000000,-9999.000000,...,Diesel,66.8,65,REST,REST,7,20,536,536,At Stop
2902,XLRTEF5300G450805,IGNITION_ON,RFMS,2024-05-24T02:59:51.265Z,2024-05-24T03:05:27.100Z,106460250,0,655350,47.660800,-2.971410,...,Diesel,0.0,65,REST,REST,20,20,2040,2040,At Stop
2903,XLRTEF5300G450805,ENGINE_ON,RFMS,2024-05-24T02:59:51.265Z,2024-05-24T03:05:27.100Z,0,0,0,47.660800,-2.971410,...,Diesel,0.0,0,None,None,0,0,0,0,At Stop
2904,XLRTEF5300G450805,DRIVER_1_WORKING_STATE_CHANGED,RFMS,2024-05-24T03:03:29.068Z,2024-05-24T03:08:23.110Z,106460255,0,0,47.660777,-2.971371,...,Diesel,66.4,0,DRIVE,DRIVER_AVAILABLE,0,0,0,0,Driving
2905,XLRTEF5300G450805,DRIVER_2_WORKING_STATE_CHANGED,RFMS,2024-05-24T03:03:29.068Z,2024-05-24T03:08:24.284Z,106460255,0,0,47.660777,-2.971371,...,Diesel,66.4,0,DRIVE,DRIVER_AVAILABLE,0,0,0,0,Driving
2906,XLRTEF5300G450805,TIMER,RFMS,2024-05-24T03:04:50.326Z,2024-05-24T03:09:46.859Z,106460625,2080,36400,47.660247,-2.972944,...,Diesel,66.0,65,DRIVE,DRIVER_AVAILABLE,7,34,1032,1032,Driving
2907,XLRTEF5300G450805,TIMER,RFMS,2024-05-24T03:09:50.329Z,2024-05-24T03:14:46.532Z,106467430,2080,38400,47.689476,-3.032709,...,Diesel,64.8,67,DRIVE,DRIVER_AVAILABLE,7,82,1112,1112,Driving
2908,XLRTEF5300G450805,TIMER,RFMS,2024-05-24T03:14:50.003Z,2024-05-24T03:19:46.638Z,106474445,2080,38400,47.733131,-3.092191,...,Diesel,64.8,67,DRIVE,DRIVER_AVAILABLE,6,85,1144,1144,Driving
2909,XLRTEF5300G450805,DISTANCE_TRAVELLED,RFMS,2024-05-24T03:14:47.600Z,2024-05-24T03:19:48.439Z,106475005,0,38400,47.737591,-3.095817,...,Diesel,64.8,67,DRIVE,None,6,83,1184,1184,Driving
